In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sqlalchemy import select
from pastiche import tables
from pastiche.database import SessionLocal

In [3]:
with SessionLocal() as db:
    result = db.query(tables.JumbleGame).first()
    print(result.jumbles[0].jumbled)

GNIES


# OpenAI exploration

In [ ]:
import openai
from pydantic import Field
from openai_function_call import OpenAISchema

In [ ]:
class JumbleClue(OpenAISchema):
    """Class representing a sentence used as a clue for a Jumble game"""
    clue_sentence: str = Field(..., description="Sentence with gaps to be filled.")
    solution: str = Field(..., description="The words to use to fill the gaps in the clue-sentence.")
    full_sentence: str = Field(..., description="The full sentence with all words.")
    
class JumbleCluesCollection(OpenAISchema):
    """List of jumble clues"""
    clues: list[JumbleClue]

In [ ]:
def generate_jumble_clues(data: str) -> JumbleCluesCollection:
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        temperature=0.1,
        functions=[JumbleCluesCollection.openai_schema],
        function_call={"name": JumbleCluesCollection.openai_schema["name"]},
        messages=[
            {
                "role": "user",
                "content": f"Consider the data below: '\n{data}' and for each entry generate the full sentence using the provided sentence with gaps (clue-sentence) and solution.",
            },
        ],
        max_tokens=1000,
    )
    return JumbleCluesCollection.from_response(completion)

In [ ]:
results = generate_jumble_clues(
    data="""
    [{'clue_sentence': 'After she learned to consistently tie her own shoes,',
    'solution': 'ITWASACINCH'},
    {'clue_sentence': 'Elvis Presley’s 1957 hit song had some people',
    'solution': 'ALLSHOOKUP'},
    {'clue_sentence': 'In 1981, this person successfully sued the “National Enquirer” for libel.',
    'solution': 'CAROL'},
    {'clue_sentence': 'When they visited Arizona’s famous canyon, the family had a',
    'solution': 'GRANDTIME'},
    {'clue_sentence': 'When Alfred Hitchcock’s movie, “The Birds,” premiered in 1963, people',
    'solution': 'FLOCKEDTOIT'}]
    """
)

In [ ]:
r = results.clues[0]

In [ ]:
[r.full_sentence for r in results.clues]

In [ ]:
sentence = "After she learned to consistently tie her own shoes, it was a cinch."

In [ ]:
67-5